In [2]:
# !pip install zstandard

In [3]:
import zstandard
import os
import json
import sys
from datetime import datetime
import logging.handlers
import pandas as pd
import io

new_list = []
wanted_keys = ("author", "subreddit", "selftext", "title")

with open("C:/Users/georg/Documents/RS_2022-08.zst", 'rb') as fh:
    dctx = zstandard.ZstdDecompressor(max_window_size=2**31)
    stream_reader = dctx.stream_reader(fh)
    text_stream = io.TextIOWrapper(stream_reader, encoding='utf-8')
    for line in text_stream:
        try:       
            obj = json.loads(line)
            if obj.get('is_self', '') == True:
                if obj.get('selftext') == '[deleted]' or obj.get('selftext') == '' or obj.get('selftext') == '[removed]' or obj.get('title') == '[deleted]' or obj.get('title') == '':
                    continue
                else:
                    new_dict = {}
                    for k in wanted_keys:
                        new_dict[k] = obj[k]
                    new_list.append(new_dict)
        except:
            continue

df = pd.DataFrame(new_list)
df.head()

,author,subreddit,selftext,title
0,Kresley,podcasts,We're lucky to have an attorney who is *also* ...,Have questions about recent Supreme Court deci...
1,erer1243,every15min,"Currently, it's July 31, 2022 at 08:00PM","Currently, it's July 31, 2022 at 08:00PM"
2,Arnadus,cryptopricesalerts,|Crypto|Number of users mentioning it|\n:--|:-...,Most trending cryptos today so far
3,YarnSwapBot,Yarnswap,Post your confirmed trades below; when confirm...,August 2022 Confirmed Trade Thread
4,Ordinary-Dog-2982,u_Ordinary-Dog-2982,인터넷릴게임 인터넷경륜 [BCGAME7.COM](https://BCGAME7.COM...,인터넷릴게임 인터넷경륜 ℡く BCGAME7.COM§ま 인터넷경륜 인터넷경륜 인터넷경륜


In [4]:
df.count()

author       5877974
subreddit    5877974
selftext     5877974
title        5877974
dtype: int64

In [6]:
filtered_df1 = df.query('selftext != "[removed]"')
filtered_df1 = filtered_df1[filtered_df1.selftext.notnull()]
filtered_df1 = filtered_df1[~filtered_df1.selftext.str.contains("bot")]
filtered_df1 = filtered_df1[~filtered_df1.selftext.str.contains("u_")]
filtered_df1 = filtered_df1.drop_duplicates()
filtered_df1 = filtered_df1.drop_duplicates(subset=['author', 'subreddit', 'title'], keep='last')

In [7]:
filtered_df1.count()

author       5026542
subreddit    5026542
selftext     5026542
title        5026542
dtype: int64

In [8]:
title_length = []
for i, title in enumerate(filtered_df1['title']):
    if type(title) != type(str(title)):
        title_length.append(0)
    else:
        title_split = title.split()
        title_length.append(len(title_split))


filtered_df1['title_length'] = title_length

invalid_title_length = []
for i, length in enumerate(filtered_df1['title_length']):
    if length < 4:
        invalid_title_length.append(1)
    elif length > 60:
        invalid_title_length.append(1)
    else:
        invalid_title_length.append(0)

filtered_df1['invalid_title_length'] = invalid_title_length

filtered_df2 = filtered_df1.query('invalid_title_length == 0')

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def create_dataframe(matrix, tokens):

    doc_names = [f'doc_{i+1}' for i, _ in enumerate(matrix)]
    df2 = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df2)

data = [filtered_df2.iloc[0].title, filtered_df2.iloc[0].selftext]

Tfidf_vect = TfidfVectorizer()
vector_matrix = Tfidf_vect.fit_transform(data)

tokens = Tfidf_vect.get_feature_names()
create_dataframe(vector_matrix.toarray(),tokens)


cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix, ['title', 'selftext'])


C:\Users\ikim1\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,title,selftext
doc_1,1.000000,0.130514
doc_2,0.130514,1.000000


In [10]:
new_df = filtered_df2.copy()

In [11]:
def cosine_sim(x):
    data = [x.title, x.selftext]
    
    try:
        vector_matrix = TfidfVectorizer().fit_transform(data)
    
    except:
        return 'check'
    
    return cosine_similarity(vector_matrix)[0,1]


new_df['cosine_sim'] = filtered_df2.apply(cosine_sim, axis=1)

In [23]:
new_df = new_df.query('cosine_sim != "check"')
new_df["cosine_sim"] = new_df["cosine_sim"].astype(float)

test_df_2 = new_df.query('cosine_sim > .1 & cosine_sim < .8')

test_df_2.count()

author                  2304934
subreddit               2304934
selftext                2304934
title                   2304934
title_length            2304934
invalid_title_length    2304934
cosine_sim              2304934
dtype: int64

In [26]:
print(f'Proportion of ineffective titles: {len(test_df_2)/len(filtered_df1)}')

Proportion of ineffective titles: 0.4585526192758361


In [30]:
test_df_2 = test_df_2.sort_values(by="cosine_sim", ascending=False)
test_df_2 = test_df_2[:20001]
test_df_2.head()
test_df_2.to_csv("C:/Users/georg/Documents/RS_2022-06-clean.csv",index=False)

,author,subreddit,selftext,title,title_length,invalid_title_length,cosine_sim
3422931,BugzzB,u_BugzzB,\n\n\#gpibs #production #company #distributio...,#gpibs #production #company #distribution The ...,25,0,0.799986
3620459,Ezy1time,PennsylvaniaNSA,SE Pa couple in need of a petite tight bodied ...,(33M/29F) seeking (f) in # Pennsylvania. Hot D...,43,0,0.799939
3462557,Impossible_Tiger722,dirtyr4r,"31 based in east, live alone and can host. Why...",31 [M4F] #east London can't sleep and wonder i...,30,0,0.799888
3462459,Impossible_Tiger722,UkHookUps,"31 based in east, live alone and can host. Why...",31 [M4F] #east London can't sleep and wonder i...,30,0,0.799888
147433,modoge7436,u_modoge7436,플레이텍바두기 플레이텍룰렛 ▶▶ＢcgＡmＥ７７７７．CΟΜ 플레이텍룰렛 ※ 플레이텍룰...,플레이텍바두기*S 플레이텍룰렛 ─▶┼─BCGAME7777.COM 플레이텍룰렛 ▥ 플...,6,0,0.799751
